<a href="https://colab.research.google.com/github/AngelR0/Binary_Classifier_Model/blob/main/model_binary_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning

## Prepare the Data

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from pathlib import Path
import numpy as np

In [2]:
# Upload creadit_card_transaction.csv
from google.colab import files
csv_file = files.upload()



Saving applicants_data.csv to applicants_data (2).csv


In [3]:
# Read the applicants_data.csv file from the Resources folder
applicants_data_df = pd.read_csv("applicants_data.csv")

# Review the DataFrame
display(applicants_data_df.head())
display(applicants_data_df.tail())

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1
34298,996086871,WATERHOUSE CHARITABLE TR,T3,Independent,C1000,Preservation,Co-operative,1,1M-5M,N,36500179,0


In [4]:
# Review the Data types associated with each columns
applicants_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Drop the `EIN` and `NAME` columns
applicants_data_df = applicants_data_df.drop(columns = ["EIN", "NAME"])

# Review the DataFrame
applicants_data_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
from pandas.core.arrays import categorical
# Create a list of categorical variables
categorical_variables = list(applicants_data_df.dtypes[applicants_data_df.dtypes == "object"].index)

# Display the variables
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a ONEHOTENCODER instance
enc = OneHotEncoder(sparse=False)

In [8]:
# Encode the categorical variables using the ONEHOTENCODER
encoded_data = enc.fit_transform(applicants_data_df[categorical_variables])

In [9]:
# Create a dataFrame with the encoded variables
encoded_df = pd.DataFrame(data = encoded_data, columns = enc.get_feature_names_out(categorical_variables))

# Display the DataFrame
encoded_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Add the numerical variables from the original DataFrame to the one_hot encoding DataFrame
encoded_df = pd.concat([applicants_data_df.drop(columns = categorical_variables), encoded_df], axis = 1)

# Display the new DataFrame
display(encoded_df.head())
display(encoded_df.tail())

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
34294,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34298,1,36500179,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["IS_SUCCESSFUL"]

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns = "IS_SUCCESSFUL")

# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split the preprocessed data into a trainning and testing datasets
# # Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training datasets
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile and Evaluate a Binary Classification Model

In [15]:
# Define the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
display(number_input_features)

116

In [16]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [17]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = (number_input_features + number_output_neurons) // 2

# Review the number hidden nodes in the first layer
display(hidden_nodes_layer1)

58

In [18]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = ( hidden_nodes_layer1 + number_output_neurons) // 2

# Display the number hidden nodes. in the second layer
display(hidden_nodes_layer2)

29

In [19]:
# Create the Sequential model instance
seq_model = Sequential()

In [20]:
# Add the first hidden layer
seq_model.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation="relu"))

In [21]:
# Add the second hidden layer
seq_model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [22]:
# Add the output layer to the model specifying the number of output neurons and activation function
seq_model.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [23]:
# Display the Sequential model summary
seq_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile and fit the model
seq_model.compile(loss= "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# Fitting using 50 epochs
seq_model.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.5754 - accuracy: 0.7196
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7281
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7296
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7311
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7337
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7349
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7317
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7333
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5435 - accuracy: 0.7348
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7346

In [25]:
# Evaluate the model loss and accuracy metrics using the evaluate method test data
model_loss, model_accuracy = seq_model.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model loss and accuracy resuls
print(f"Loss: {model_loss} \nAccuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5575 - accuracy: 0.7291 - 489ms/epoch - 2ms/step
Loss: 0.5574955344200134 
Accuracy: 0.7290962338447571


In [50]:
# Save the models file path
file_path = "../Resources/AlphabetSoup.h5"

# Export your model to a HDF5 file
seq_model.save(file_path)

## Optimize the Neural Network Model

### Alternative Model 1

In [27]:
# Define the number of inputs
num_input_features = len(X_train.iloc[0])

# Review the number of features
display(num_input_features)

116

In [28]:
# Define the number of neurons in the output layer
num_output_neurons_A1 = 1

In [29]:
# Define the number of hidden nodes for the first hidden layer
A1_hidden_nodes_layer1 = (num_input_features + num_output_neurons_A1) // 1.5

# Review the number of hidden nodes in the first layer
A1_hidden_nodes_layer1

78.0

In [30]:
# Create the Sequential model instance
A1_seq_model = Sequential()

In [31]:
# First hidden layer
A1_seq_model.add(Dense(units = A1_hidden_nodes_layer1, input_dim= num_input_features, activation="relu"))

# Output layer
A1_seq_model.add(Dense(units=num_output_neurons_A1, activation="sigmoid"))

# Check the structure of the model 
A1_seq_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 78)                9126      
                                                                 
 dense_4 (Dense)             (None, 1)                 79        
                                                                 
Total params: 9,205
Trainable params: 9,205
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the Sequential Model
A1_seq_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Fit the model using 100 epochs
fit_modelA1 = A1_seq_model.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5977 - accuracy: 0.7155
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5568 - accuracy: 0.7298
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7306
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5495 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7321
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7315
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accura

### Alternative Model 2

In [34]:
# Define the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Display the number of features 
number_input_features

116

In [35]:
# Define the number of neurons in the output Layer
num_output_neurons_A2 = 1

In [36]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer2 = (number_input_features + num_output_neurons_A2)

# Review the number of hidden nodes in the first layer
hidden_nodes_layer2

117

In [37]:
# Create the Sequential model instance
A2_seq_model = Sequential()

In [38]:
# First hidden layer
A2_seq_model.add(Dense(units = hidden_nodes_layer2, input_dim = number_input_features, activation="relu"))

# Output layer
A2_seq_model.add(Dense(units = num_output_neurons_A2, activation="sigmoid"))

# Check the structure of the model
A2_seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 117)               13689     
                                                                 
 dense_6 (Dense)             (None, 1)                 118       
                                                                 
Total params: 13,807
Trainable params: 13,807
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile and fit the model
A2_seq_model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])

A2_seq_model.fit(X_train_scaled, y_train, epochs = 150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5767 - accuracy: 0.7211
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5592 - accuracy: 0.7285
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5551 - accuracy: 0.7296
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7304
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7319
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accuracy: 0.7308
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7303
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7324
Epoch 9/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5465 - accuracy: 0.7313
Epoch 10/150
804/804 [==============================] - 4s 5ms/step - loss: 0.5463 - accura

In [40]:
# Display the accuracy scores achieved by each model and compare the results

print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the 'evaluate' method and test data
model_loss, model_accuracy = seq_model.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model and accuracy results 
print(f"Loss: {model_loss} \nAccuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5575 - accuracy: 0.7291 - 352ms/epoch - 1ms/step
Loss: 0.5574955344200134 
Accuracy: 0.7290962338447571


In [41]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the 'evaluate' method and test data
model_loss, model_accuracy = A1_seq_model.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model and accuracy results 
print(f"Loss: {model_loss} \nAccuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.5586 - accuracy: 0.7297 - 444ms/epoch - 2ms/step
Loss: 0.5586122274398804 
Accuracy: 0.72967928647995


In [42]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the 'evaluate' method and test data
model_loss, model_accuracy = A2_seq_model.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model and accuracy results 
print(f"Loss: {model_loss} \nAccuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.6085 - accuracy: 0.7321 - 444ms/epoch - 2ms/step
Loss: 0.6084644198417664 
Accuracy: 0.7321282625198364


In [43]:
# Set the file path for the first alternative model
file_path = Path("../Resources/AlphabetSoup_A1.h5")

# Export your model to a HDF5 file
A1_seq_model.save(file_path)

In [47]:
# Set the file path for the first alternative model
file_path = Path("Resources/AlphabetSoup_A2.h5")

# Export your model to a HDF5 file
A2_seq_model.save(file_path)